# test create date

In [1]:
from faker import Faker
n=5
f = Faker()
[f.date_between(start_date='-5y',end_date='-1d') for i in range(n)]

[datetime.date(2018, 11, 12),
 datetime.date(2020, 4, 20),
 datetime.date(2021, 12, 10),
 datetime.date(2023, 6, 7),
 datetime.date(2023, 2, 24)]

In [2]:
import pyarrow as pa
import pyarrow.parquet
import pyarrow.flight
# client = pa.flight.connect(location='grpc://192.168.24.90:8816')
client = pa.flight.connect(location='grpc://0.0.0.0:8815')

# create df

In [3]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 62.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40763,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 62.52 GiB
Comm: tcp://127.0.0.1:41375,Total threads: 2
Dashboard: http://127.0.0.1:39003/status,Memory: 15.63 GiB
Nanny: tcp://127.0.0.1:43951,


In [4]:
import dask.dataframe as dd
import numpy as np
import rstr

n=int(1e7)
i=list(np.arange(n))
ddf = dd.DataFrame.from_dict({'a':i}, npartitions=n//1024+1)
ddf = ddf.map_partitions(lambda df: df.assign(b=[ rstr.xeger(r'([A-Z]\d{20})') for i in range(len(df))]))
ddf = ddf.map_partitions(lambda df: df.assign(c=[ rstr.xeger(r'([A-Z]\d{10})') for i in range(len(df))]))
ddf = ddf.map_partitions(lambda df: df.assign(date=[ f.date_between(start_date='-3d',end_date='-0d') for i in range(len(df))]))


In [ ]:
ddf.compute()


In [ ]:
import rstr
import pandas as pd

n=int(1e1)
d={
    'a':[i**2 for i in range(n)],
    'b':[rstr.xeger(r'([A-Z]\d{20})') for i in range(n)],
    'c':[rstr.xeger(r'([A-Z]\d{10})') for i in range(n)],
    'date':[f.date_between(start_date='-3d',end_date='-0d') for i in range(n)]
}
table=pa.table(d)
df=pd.DataFrame(d)
df

# create a new repo

In [ ]:
import os
import shutil
def reset_dir(path='datasets'):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

reset_dir()

In [ ]:
!tree datasets -la

# test splitting batch and writting to a parquet

In [ ]:
%%time
# reader = table.to_batches(max_chunksize=2)
reader = table.to_batches()
with open("datasets/test.parquet","wb") as sink:
    with pa.parquet.ParquetWriter(sink, table.schema) as writer:
        for chunk in reader:
            writer.write_table(pa.Table.from_batches([chunk]))

In [ ]:
!tree datasets -la

# Write df a dataset contains many parquets

In [ ]:
import pyarrow.dataset as ds
import datetime

In [ ]:
%%time
table = pa.Table.from_pandas(df)
date_schema=schema=table.schema.remove(0).remove(0).remove(0)
part = ds.partitioning(date_schema, flavor="hive",)

repo='datasets'
table_name='df'

ds.write_dataset(
    data=table,
    base_dir=os.path.join(repo,table_name),
    format="parquet",
    partitioning=part,
#     max_rows_per_file=3,
#     max_rows_per_group=2,
    basename_template = "part-{:%y%m%d-%H%M%S}-{{i}}.parquet".format(datetime.datetime.now()),
#     existing_data_behavior='delete_matching',
    existing_data_behavior='overwrite_or_ignore',
)

In [ ]:
!tree datasets

In [ ]:
%%time
import duckdb
duckdb.sql("SELECT * FROM 'datasets/df/*/*.parquet';").show()

# Second write append rows

In [ ]:
%%time
import pyarrow.dataset as ds
import datetime
import time

time.sleep(1)
repo='datasets'
table_name='df'


df['last_updated']=str(datetime.datetime.now() )
table = pa.Table.from_pandas(df)
date_schema=schema=table.schema.remove(0).remove(0).remove(0).remove(1)
part = ds.partitioning(date_schema, flavor="hive",)

ds.write_dataset(
    data=table,
    base_dir=os.path.join(repo,table_name),
    format="parquet",
    partitioning=part,
#     max_rows_per_file=3,
#     max_rows_per_group=2,
    basename_template = "part-{:%y%m%d-%H%M%S}-{{i}}.parquet".format(datetime.datetime.now()),
#     existing_data_behavior='delete_matching',
    existing_data_behavior='overwrite_or_ignore',
)

In [ ]:
import duckdb
duckdb.sql("""
SELECT * FROM read_parquet('datasets/df/*/*.parquet',union_by_name=True);
""").show()

# Third write append rows

In [ ]:
import pyarrow.dataset as ds
import datetime
import time

time.sleep(1)
repo='datasets'
table_name='df'
part = ds.partitioning(date_schema, flavor="hive",)

df['last_updated']=str(datetime.datetime.now() )
table = pa.Table.from_pandas(df)
date_schema=schema=table.schema.remove(0).remove(0).remove(0).remove(1)
ds.write_dataset(
    data=table,
    base_dir=os.path.join(repo,table_name),
    format="parquet",
    partitioning=part,
#     max_rows_per_file=3,
#     max_rows_per_group=2,
    basename_template = "part-{:%y%m%d-%H%M%S}-{{i}}.parquet".format(datetime.datetime.now()),
#     existing_data_behavior='delete_matching',
    existing_data_behavior='overwrite_or_ignore',
)

In [ ]:
import duckdb
duckdb.sql("""
SELECT * FROM read_parquet('datasets/df/*/*.parquet',union_by_name=True);
""").show()

In [ ]:
import duckdb
duckdb.sql("""
SELECT count(a) FROM read_parquet('datasets/df/*/*.parquet',union_by_name=True);
""").show()

```python
import pyarrow as pa
import pyarrow.flight

# client = pa.flight.connect("grpc://0.0.0.0:8815")
client = pa.flight.connect(location='grpc://192.168.24.90:8816')

# Upload a new dataset
NUM_BATCHES = 1024
ROWS_PER_BATCH = 4096
upload_descriptor = pa.flight.FlightDescriptor.for_path("streamed.parquet")
writer, _ = client.do_put(upload_descriptor, table.schema)
with writer:
    for batch in table.to_batches(max_chunksize=2):
        writer.write_batch(batch)
```